<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
# !pip3 install folium
# !pip3 install wget
# !pip3 install pandas

In [2]:
%reset -f
# import folium
import pandas as pd
import numpy as np

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon
from folium import Popup, CircleMarker, Map, Marker, Icon, PolyLine
import folium

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [4]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(spacex_csv_file)

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


## Task 1: Mark all launch sites on a map


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


These are the coordinates for each site.


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
# sites_map = Map(location=nasa_coordinate, zoom_start=10)

Now, let's add a circle for each launch site in data frame `launch_sites`


In [7]:
# Initial the map
sites_map = Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for _, site in launch_sites_df.iterrows():
    launch_site = site['Launch Site']
    location = [site['Lat'], site['Long']]
    popup = Popup(launch_site)
    radius = 50
    toolip = "Click me and zoom in."
    CircleMarker(location, radius, popup, toolip, fill=True).add_to(sites_map)

    ic = Icon(icon = 'info-sign')
    Marker(location, icon=ic).add_to(sites_map)

sites_map

Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Launch sites are not in proximity to the Equator line.
- Launch sites are in close proximity to the coast.

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [8]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [9]:
marker_cluster = MarkerCluster()


Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [10]:
# Just use numpy.
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df['marker_color'] = np.where(spacex_df['class'] == 1, 'green', 'red')
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


For each launch result in `spacex_df` data frame, we add a `folium.Marker` to `marker_cluster`


In [11]:
# Add marker_cluster to current sites_map
sites_map.add_child(marker_cluster)

# for each row in spacex_df data frame # create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
for _, row in spacex_df.iterrows():

    icon_background_color = 'white'
    marker_color = row['marker_color']
    rocket_icon = Icon(color=icon_background_color, icon_color=marker_color, icon='rocket', prefix='fa')
    location = [row['Lat'], row['Long']]

    marker = folium.Marker(location, icon=rocket_icon)
    marker_cluster.add_child(marker)

sites_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [12]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to calculate the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [17]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(locations):
    lat1, lon1, lat2, lon2 = locations[0][0], locations[0][1], locations[1][0], locations[1][1]
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [14]:
# Find coordinate of the closet coastline
cca_loc = [28.56323, -80.57688]
cca_map = Map(location=cca_loc, zoom_start=15).add_child(mouse_position)

In [ ]:
def draw_line(destination):
    locations = [ cca_loc, destination ]

    weight = 4
    color = "#FF0000"

    PolyLine(locations, weight, color).add_to(cca_map)
    distance = calculate_distance(locations)
    Marker(
        destination,
        icon=DivIcon(
                icon_size=(100,100),
                icon_anchor=(0,0),
                html='<div style="font-size: 40; color:#FF0000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance))
        ).add_to(cca_map)

In [ ]:
# To coastline
coastline =  [28.56342, -80.56750]


# To highway
highway = [28.411780, -80.820600]

# To orlando
city = [28.5383, -81.37900]

destinations = [coastline, highway, city]

for _, destination in enumerate(destinations):
    draw_line(destination)

cca_map

After you plot distance lines to the proximities, you can answer the following questions easily:
- There are no launch sites in close proximity to railways.
- There are no launch sites in close proximity to highways.
- Launch sites are in close proximity to coastline.
- Launch sites are located some distance away from cities.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
